In [1]:
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tsfel

# Pre-processing and feature engineering

### Preprocessing raw training data

#### Read in raw data

In [15]:
big_df = pd.read_csv('./valdata.csv', header=0, index_col=0)
big_df

,Row,Col,OD,GFP,Cycle,Time,Condition,Plate,Strain
0,A,1,0.1910,13,1,0.000000,1,1,0
1,A,2,0.1868,12,1,0.000000,2,1,0
2,A,3,0.1937,12,1,0.000000,3,1,0
3,A,4,0.1914,12,1,0.000000,4,1,0
4,A,5,0.1975,13,1,0.000000,5,1,0
...,...,...,...,...,...,...,...,...,...
122881,H,5,0.3715,18,96,21.914568,101,5,14
122882,H,4,0.3920,17,96,21.914568,100,5,14
122883,H,3,0.3986,19,96,21.914568,99,5,14
122884,H,2,0.3826,19,96,21.914568,98,5,14


#### Read in control data for each strain (in train_preprocessing) 

In [3]:
ctrl_gfp = pd.read_csv("./ctrl_gfp.csv",header=0, index_col=0)
ctrl_gfp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,11.666667,12.666667,22.000000,13.333333,12.333333,31.000000,14.333333,11.000000,20.666667,30.333333,11.666667,74.333333,12.333333,16.333333,25.666667
1,11.666667,12.666667,22.666667,14.666667,13.000000,33.666667,17.000000,11.000000,22.666667,29.666667,11.333333,76.000000,12.333333,16.333333,29.000000
2,11.000000,13.000000,24.666667,14.666667,13.333333,35.666667,20.666667,11.000000,24.000000,31.000000,10.000000,80.000000,12.000000,16.000000,32.333333
3,12.000000,13.333333,27.333333,15.666667,13.333333,40.333333,25.000000,11.333333,25.000000,33.000000,10.333333,80.000000,12.666667,15.666667,38.333333
4,12.000000,13.333333,31.000000,17.333333,14.000000,48.333333,31.666667,11.333333,26.666667,36.666667,10.666667,81.666667,12.000000,16.000000,43.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,22.333333,NaN,NaN,23.333333,23.000000,NaN,NaN,17.000000,23.333333,NaN,NaN,207.666667,32.000000,29.666667,NaN
91,22.333333,NaN,NaN,24.000000,23.000000,NaN,NaN,16.666667,23.666667,NaN,NaN,208.333333,31.333333,30.333333,NaN
92,22.666667,NaN,NaN,24.666667,23.333333,NaN,NaN,17.000000,23.000000,NaN,NaN,208.333333,32.000000,29.666667,NaN
93,22.666667,NaN,NaN,24.000000,23.000000,NaN,NaN,16.666667,22.666667,NaN,NaN,209.000000,32.333333,NaN,NaN


#### Subtract control gfp for all strains

In [16]:
#Truncated to 10.4 hours for everything
big_df_trunc = big_df.loc[big_df['Time'] <= 10.4]
big_df_trunc = big_df_trunc.dropna(axis=0)

for i in big_df_trunc.index:
    strain = big_df_trunc.loc[i,'Strain']
    cycle = big_df_trunc.loc[i, 'Cycle']
    big_df_trunc.loc[i, 'GFP'] = big_df_trunc.loc[i, 'GFP'] - ctrl_gfp.loc[cycle - 1, str(strain)]
    
big_df_trunc

/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/459027639.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.333333333333334' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  big_df_trunc.loc[i, 'GFP'] = big_df_trunc.loc[i, 'GFP'] - ctrl_gfp.loc[cycle - 1, str(strain)]


,Row,Col,OD,GFP,Cycle,Time,Condition,Plate,Strain
0,A,1,0.1910,1.333333,1,0.000000,1,1,0
1,A,2,0.1868,0.333333,1,0.000000,2,1,0
2,A,3,0.1937,0.333333,1,0.000000,3,1,0
3,A,4,0.1914,0.333333,1,0.000000,4,1,0
4,A,5,0.1975,1.333333,1,0.000000,5,1,0
...,...,...,...,...,...,...,...,...,...
104731,H,5,0.3506,-27.333333,46,10.380587,101,5,14
104732,H,4,0.3678,-28.333333,46,10.380587,100,5,14
104733,H,3,0.3816,-26.333333,46,10.380587,99,5,14
104734,H,2,0.3682,-27.333333,46,10.380587,98,5,14


### Feature engineering

#### TSFELS package

In [17]:
########## ONLY RUN THIS IF YOU WANT TO REGENERATE THE TSFELS FEATURES FOR EACH STRAIN. ##########
########## CSVS ARE PROVIDED IN FOLDER './tsfel' ##########
import re

strains = list(range(0,15,1))
conditions = list(range(1,109,1))
new_columns = list(range(1,45,1))

# For each strain, build a table of GFP time traces (rows = conditions, columns = cycles)
for i in strains:
    new_df = pd.DataFrame(columns=new_columns)

    for j in conditions:
        # Extract GFP values for this strain/condition pair (up to cycle 44)
        new_row = big_df_trunc.loc[
            (big_df_trunc['Strain'] == i) &
            (big_df_trunc['Condition'] == j)
        ].T.loc['GFP'][:44]

        # Convert row to a single-row DataFrame indexed by condition
        df = pd.DataFrame([new_row], index=[j])
        df.columns = new_columns

        # Append the condition row to the strain-specific time-trace matrix
        new_df = pd.concat([new_df, df], ignore_index=False)

    # Once new_df is complete, compute TSFEL features for each condition
    all_feats = pd.DataFrame()

    # Loop across each condition (column after transposing)
    for j in new_df.T.columns:
        df = pd.DataFrame(new_df.T[j])

        # Extract full feature set using TSFEL
        cfg_file = tsfel.get_features_by_domain()
        df_feats = tsfel.time_series_features_extractor(cfg_file, df)

        # Rename row index to match the condition ID
        df_feats.index = [j]

        # Clean up column names (remove leading "cond_x_")
        new_index = []
        for k in df_feats.columns:
            match = re.search(r'\d+_(.*)', k)
            if match:
                substring = match.group(1)
            new_index.append(substring)
        df_feats.columns = new_index

        # Merge into master feature matrix (features × conditions)
        if all_feats.empty:
            all_feats = df_feats.T
        else:
            all_feats = all_feats.join(df_feats.T)

    # Save features for this strain
    all_feats.to_csv('tsfel_feats_strain'+str(i)+'.csv')

*** Feature extraction started ***


/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/3614378367.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, df], ignore_index=False)
/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/3614378367.py:36: UserWarning: Using default sampling frequency set in configuration file.
  df_feats = tsfel.time_series_features_extractor(cfg_file, df)



*** Feature extraction finished ***
*** Feature extraction started ***


/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/3614378367.py:36: UserWarning: Using default sampling frequency set in configuration file.
  df_feats = tsfel.time_series_features_extractor(cfg_file, df)



*** Feature extraction finished ***
*** Feature extraction started ***


...


*** Feature extraction finished ***
*** Feature extraction started ***


/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/3614378367.py:36: UserWarning: Using default sampling frequency set in configuration file.
  df_feats = tsfel.time_series_features_extractor(cfg_file, df)



*** Feature extraction finished ***
*** Feature extraction started ***


/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/3614378367.py:36: UserWarning: Using default sampling frequency set in configuration file.
  df_feats = tsfel.time_series_features_extractor(cfg_file, df)



*** Feature extraction finished ***


#### Get minimum and skewness features 

In [18]:
########## ONLY RUN THIS IF YOU WANT TO REGENERATE df_Min.csv and df_Skewness.csv ##########
########## CSVS ARE PROVIDED IN FOLDER './val' ##########


dfs = {}

# Loop through each CSV file
for i in range(0,15,1):

    filename = './tsfel/tsfel_feats_strain'+f'{i}.csv'
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filename, index_col=0)
    
    # Store the DataFrame in the dictionary with a key based on the file number
    dfs[f'strain_{i}'] = df

#add column with strain #
strain_num_list = []

for i in dfs: 
    strain_num = i
    strain_num_list = [strain_num] * len(dfs[i])
    dfs[i]['strain'] = strain_num_list
    
#concatenate all dataframes into one big df concat_df
concat_df = pd.DataFrame()

for i in dfs: 
    concat_df = pd.concat([concat_df, dfs[i]])
    
strains = list(range(0,15,1))

for i in ['Min', 'Skewness']:
    df_new = pd.DataFrame()
    df_new = concat_df.loc[i].T
    df_new.columns = strains
    df_new = df_new.drop(['strain'])
    df_new.to_csv('df_'+i+'.csv')

#### Get maximum feature

In [19]:
########## ONLY RUN THIS IF YOU WANT TO REGENERATE df_fmax.csv ##########
########## CSVS ARE PROVIDED IN FOLDER './train' ##########

strains = np.unique(big_df_trunc['Strain'].values)
conditions = list(range(1,109,1))

df_fmax = pd.DataFrame(0, columns=pd.Index(strains, name='strains'), index=pd.Index(conditions, name='conditions'))

#for each strain
for i in strains:
    #for each condition (1-108)
    for j in conditions:
        #get GFP_max 
        fmax = big_df_trunc.loc[big_df_trunc['Strain']==i].loc[big_df_trunc['Condition']==j].loc[big_df_trunc['Time'] <= 10.4]['GFP'].max()        
        df_fmax.loc[j, i] = fmax

df_fmax.to_csv('df_fmax.csv')

/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/1767978273.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '674.6666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_fmax.loc[j, i] = fmax
/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/1767978273.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '62.66666666666667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_fmax.loc[j, i] = fmax
/var/folders/r4/n0pt1c0x7ql2s2xvp0l_fsl00000gn/T/ipykernel_6942/1767978273.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '87.66666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype fi